In [ ]:
!pip install torch torchvision diffusers transformers accelerate safetensors pillow

In [ ]:
!pip install fastapi uvicorn pyngrok nest-asyncio

In [ ]:
!pip install gradio

In [ ]:
!pip install langdetect python-dotenv

In [ ]:
import os
from pyngrok import ngrok
from fastapi import FastAPI
from pydantic import BaseModel
from langdetect import detect
from starlette.responses import FileResponse
import torch, gc
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline
from PIL import Image
import nest_asyncio
import asyncio
import uvicorn
from fastapi.responses import HTMLResponse

In [ ]:
# 환경 변수 설정

os.environ['HF_TOKEN'] = 'HUGGINGFACE TOKEN 값'
os.environ['MODEL_ID_LLAMA'] = 'meta-llama/Meta-Llama-3-8B-Instruct'
os.environ['MODEL_ID_SD'] = 'runwayml/stable-diffusion-v1-5'
os.environ['DEVICE'] = 'cuda'

HF_TOKEN = os.environ['HF_TOKEN']
MODEL_ID_LLAMA = os.environ['MODEL_ID_LLAMA']
MODEL_ID_SD = os.environ['MODEL_ID_SD']
DEVICE = os.environ['DEVICE']

NGROK_AUTH_TOKEN = "NGROK TOKEN 값"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)


In [ ]:
# 유틸 함수
def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def get_env(name, default=None):
    return os.getenv(name, default)

In [ ]:
class LlamaTranslator:
    def __init__(self):
        clear_memory()
        self.model_id = MODEL_ID_LLAMA
        self.token = HF_TOKEN
        self.offload_dir = "offload_8b"
        os.makedirs(self.offload_dir, exist_ok=True)

        self.tokenizer = AutoTokenizer.from_pretrained(self.model_id, use_auth_token=self.token)
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_id,
            use_auth_token=self.token,
            device_map="auto",
            offload_folder=self.offload_dir,
            torch_dtype=torch.float16,
            trust_remote_code=True
        )

    def generate(self, system_message, user_message):
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ]
        input_ids = self.tokenizer.apply_chat_template(
            messages, add_generation_prompt=True, return_tensors="pt"
        ).to(self.model.device)

        eot_id = self.tokenizer.convert_tokens_to_ids("<|eot_id|>")
        terminators = [self.tokenizer.eos_token_id]
        if eot_id is not None:
            terminators.append(eot_id)

        outputs = self.model.generate(
            input_ids,
            max_new_tokens=256,
            eos_token_id=terminators if len(terminators) > 1 else terminators[0],
            do_sample=True,
            temperature=0.6,
            top_p=0.9
        )

        response = outputs[0][input_ids.shape[-1]:]
        return self.tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
class ImageGenerator:
    def __init__(self):
        self.model_id = MODEL_ID_SD
        self.device = DEVICE

    def generate_image(self, prompt, height=512, width=512, guidance_scale=8):
        clear_memory()
        pipe = StableDiffusionPipeline.from_pretrained(
            self.model_id,
            torch_dtype=torch.float16,
            safety_checker=None
        ).to(self.device)
        pipe.enable_attention_slicing()
        pipe.enable_vae_slicing()

        with torch.autocast("cuda"):
            result = pipe(prompt, height=height, width=width, guidance_scale=guidance_scale)

        image = result.images[0]
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_name = f"generated_image_{timestamp}.jpg"
        image.save(file_name)

        del pipe
        clear_memory()
        return image, file_name

In [ ]:
app = FastAPI(title="AI Image Generator API", version="1.0")

class PromptRequest(BaseModel):
    prompt_text: str

# HTML 폼 제공
@app.get("/generate", response_class=HTMLResponse)
def generate_form():
    html_content = """
    <html>
        <head>
            <title>AI Image Generator</title>
        </head>
        <body>
            <h2>AI Image Generator</h2>
            <form id="generate-form">
                <label>Prompt:</label><br>
                <input type="text" id="prompt_text" size="50"><br><br>
                <button type="button" onclick="submitPrompt()">Generate Image</button>
            </form>
            <h3>Result:</h3>
            <div id="result"></div>

            <script>
                async function submitPrompt() {
                    const prompt = document.getElementById("prompt_text").value;
                    const response = await fetch("/api/generate", {
                        method: "POST",
                        headers: {"Content-Type": "application/json"},
                        body: JSON.stringify({prompt_text: prompt})
                    });
                    const data = await response.json();
                    document.getElementById("result").innerHTML = 
                        "<b>Translated Prompt:</b> " + data.translated_prompt + "<br>" +
                        "<b>Image Path:</b> <a href='" + data.image_path + "' target='_blank'>View Image</a>";
                }
            </script>
        </body>
    </html>
    """
    return html_content

@app.post("/api/generate")
def generate_image_api(req: PromptRequest):
    prompt_text = req.prompt_text
    try:
        lang = detect(prompt_text)
    except Exception:
        lang = "unknown"

    translator = LlamaTranslator()

    # 한국어 입력 시: 번역 + 세부 묘사 강화
    if lang == "ko":
        system_message = (
            "You are a professional prompt engineer and creative translator. "
            "When given a Korean description, translate it into a vivid and imaginative English prompt "
            "that expands the original meaning with rich visual details, atmosphere, lighting, and artistic style. "
            "Optimize every prompt for high-quality wallpaper creation in Stable Diffusion."
        )
        translated_prompt = translator.generate(system_message, prompt_text)

    # 영어 입력 시: 번역 없이 wallpaper 스타일로 확장
    else:
        system_message = (
            "You are a professional Stable Diffusion prompt engineer. "
            "Enhance the given English prompt into a natural and visually rich version "
            "suitable for a cinematic, high-quality wallpaper. "
            "Add realistic details about lighting, color tones, composition, and atmosphere "
            "while keeping the original meaning intact."
        )
        translated_prompt = translator.generate(system_message, prompt_text)

    generator = ImageGenerator()
    image, file_path = generator.generate_image(translated_prompt)

    return {
        "original_prompt": prompt_text,
        "translated_prompt": translated_prompt,
        "image_path": file_path
    }


In [ ]:
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

In [ ]:
config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)

asyncio.get_event_loop().run_until_complete(server.serve())